In [1]:
import numpy as np
import pandas as pd
import kaiwu as kw

# FOSSP 实例参数
n, m = 3, 3  # 作业数，机器数
l = 2  # 二进制位数
P = np.array([[1, 3, 3], [3, 1, 3], [3, 3, 1]])  # 加工时间矩阵
K1, K2 = 10000, 10000  # 惩罚系数
# 启用详细日志输出
# kw.utils.set_log_level("INFO")
# 总变量数
N = n * m + l + m * l  # 17 个自旋

# 初始化 QUBO 矩阵
Q = np.zeros((N, N))

# 项 1：最小化 C_max
for h in range(l):
    k = n * m + h
    Q[k, k] += 2 ** h

# 项 2：约束 sum_j x_ij = 1
for i in range(n):
    for j in range(m):
        idx_j = i * m + j
        Q[idx_j, idx_j] += -K1
        for k in range(j + 1, m):
            idx_k = i * m + k
            Q[idx_j, idx_k] += K1
            Q[idx_k, idx_j] += K1
# 项 3：机器完工时间约束
for j in range(m):
    x_indices = [i * m + j for i in range(n)]
    tau_start = n * m + l
    tau_indices = [tau_start + j * l + h for h in range(l)]
    z_indices = [n * m + h for h in range(l)]
    V_j = x_indices + tau_indices + z_indices
    c = {}
    for idx in x_indices:
        i = idx // m
        c[idx] = P[i, j]
    for h in range(l):
        idx_tau = tau_indices[h]
        c[idx_tau] = 2 ** h
        idx_z = z_indices[h]
        c[idx_z] = -2 ** h
    for v_a in V_j:
        Q[v_a, v_a] += K2 * c[v_a] ** 2
    for a in range(len(V_j)):
        for b in range(a + 1, len(V_j)):
            v_a = V_j[a]
            v_b = V_j[b]
            coeff = K2 * c[v_a] * c[v_b]
            Q[v_a, v_b] += coeff
            Q[v_b, v_a] += coeff

In [3]:
# 验证 QUBO 矩阵
print(f"QUBO 矩阵形状: {Q.shape}")
print(f"QUBO 矩阵是否对称: {np.allclose(Q, Q.T)}")
print(f"QUBO 矩阵非零元素数: {np.count_nonzero(Q)}")

QUBO 矩阵形状: (17, 17)
QUBO 矩阵是否对称: True
QUBO 矩阵非零元素数: 154


In [4]:
# 保存 QUBO 矩阵
labels = [f"x_{i + 1}{j + 1}" for i in range(n) for j in range(m)] + \
         [f"z_{h}" for h in range(l)] + \
         [f"tau_{j + 1}{h}" for j in range(m) for h in range(l)]
Q_df = pd.DataFrame(Q, index=labels, columns=labels)
Q_df.to_csv("fossp_qubo.csv")
print("QUBO 矩阵已保存为 fossp_qubo.csv")

QUBO 矩阵已保存为 fossp_qubo.csv


In [5]:
# 转换为 Ising 模型
R = np.sum(Q, axis=1)
h = -R / 2
J_temp = np.triu(-Q / 4, k=1)
J = J_temp + J_temp.T
print(J)

[[     0.  -2500.  -2500.  -7500.      0.      0.  -7500.      0.      0.
    2500.   5000.  -2500.  -5000.      0.      0.      0.      0.]
 [ -2500.      0.  -2500.      0.  -7500.      0.      0. -22500.      0.
    7500.  15000.      0.      0.  -7500. -15000.      0.      0.]
 [ -2500.  -2500.      0.      0.      0. -22500.      0.      0.  -7500.
    7500.  15000.      0.      0.      0.      0.  -7500. -15000.]
 [ -7500.      0.      0.      0.  -2500.  -2500. -22500.      0.      0.
    7500.  15000.  -7500. -15000.      0.      0.      0.      0.]
 [     0.  -7500.      0.  -2500.      0.  -2500.      0.  -7500.      0.
    2500.   5000.      0.      0.  -2500.  -5000.      0.      0.]
 [     0.      0. -22500.  -2500.  -2500.      0.      0.      0.  -7500.
    7500.  15000.      0.      0.      0.      0.  -7500. -15000.]
 [ -7500.      0.      0. -22500.      0.      0.      0.  -2500.  -2500.
    7500.  15000.  -7500. -15000.      0.      0.      0.      0.]
 [     0. -22

In [6]:
# 验证 Ising 参数
print(f"J 矩阵形状: {J.shape}")
print(f"h 向量长度: {len(h)}")

J 矩阵形状: (17, 17)
h 向量长度: 17


In [7]:
# 创建基础优化器（模拟退火）
base_optimizer = kw.classical.SimulatedAnnealingOptimizer(
    initial_temperature=100,
    alpha=0.99,
    cutoff_temperature=0.001,
    iterations_per_t=10,
    size_limit=5
)

In [12]:
Q1 = kw.preprocess.adjust_ising_matrix_precision(Q)
print(Q1)
Q2 = pd.DataFrame(Q1, index=labels, columns=labels)
Q2.to_csv("fossp_qubo_q2.csv")
print("QUBO 矩阵已保存为 fossp_qubo.csv")

[[  0  11  11  32   0   0  32   0   0 -11 -21  11  21   0   0   0   0]
 [ 11  85  11   0  32   0   0  95   0 -32 -63   0   0  32  63   0   0]
 [ 11  11  85   0   0  95   0   0  32 -32 -63   0   0   0   0  32  63]
 [ 32   0   0  85  11  11  95   0   0 -32 -63  32  63   0   0   0   0]
 [  0  32   0  11   0  11   0  32   0 -11 -21   0   0  11  21   0   0]
 [  0   0  95  11  11  85   0   0  32 -32 -63   0   0   0   0  32  63]
 [ 32   0   0  95   0   0  85  11  11 -32 -63  32  63   0   0   0   0]
 [  0  95   0   0  32   0  11  85  11 -32 -63   0   0  32  63   0   0]
 [  0   0  32   0   0  32  11  11   0 -11 -21   0   0   0   0  11  21]
 [-11 -32 -32 -32 -11 -32 -32 -32 -11  32  63 -11 -21 -11 -21 -11 -21]
 [-21 -63 -63 -63 -21 -63 -63 -63 -21  63 127 -21 -42 -21 -42 -21 -42]
 [ 11   0   0  32   0   0  32   0   0 -11 -21  11  21   0   0   0   0]
 [ 21   0   0  63   0   0  63   0   0 -21 -42  21  42   0   0   0   0]
 [  0  32   0   0  11   0   0  32   0 -11 -21   0   0  11  21   0   0]
 [  0 

In [8]:
# 添加精度降级装饰器（目标精度8位）
precision_optimizer = kw.preprocess.adjust_ising_matrix_precision(base_optimizer, precision=8)
kw.preprocess.perform_precision_adaption_mutate()
# 执行求解
solution = precision_optimizer.solve(J)

AttributeError: module 'kaiwu.preprocess' has no attribute 'PrecisionReducer'

In [40]:
# # 转换为伊辛模型
# J = np.zeros((N, N))
# h = np.zeros(N)
# for i in range(N):
#     h[i] = Q[i, i] / 4 + sum(Q[i, j] / 4 for j in range(N) if j != i)
#     for j in range(i + 1, N):
#         J[i, j] = Q[i, j] / 4
# J = J + J.T
# h += np.sum(Q, axis=1) / 4

In [41]:
# # 验证 QUBO 矩阵
# print(f"QUBO 矩阵形状: {Q.shape}")
# print(f"QUBO 矩阵是否对称: {np.allclose(Q, Q.T)}")
# print(f"QUBO 矩阵非零元素数: {np.count_nonzero(Q)}")
# print("QUBO 矩阵前 5 行 5 列:\n", Q[:5, :5])
#
# # 保存 QUBO 矩阵到 CSV
# labels = [f"x_{i + 1}{j + 1}" for i in range(n) for j in range(m)] + \
#          [f"z_{h}" for h in range(l)] + \
#          [f"tau_{j + 1}{h}" for j in range(m) for h in range(l)]
# Q_df = pd.DataFrame(Q, index=labels, columns=labels)
# Q_df.to_csv("qubo_matrix_corrected.csv")
# print("标准 QUBO 矩阵已保存为 qubo_matrix_corrected.csv")

QUBO 矩阵形状: (17, 17)
QUBO 矩阵是否对称: True
QUBO 矩阵非零元素数: 29
QUBO 矩阵前 5 行 5 列:
 [[-20000.  20000.  20000.      0.      0.]
 [ 20000. -20000.  20000.      0.      0.]
 [ 20000.  20000. -20000.      0.      0.]
 [     0.      0.      0. -20000.  20000.]
 [     0.      0.      0.  20000. -20000.]]
标准 QUBO 矩阵已保存为 qubo_matrix_corrected.csv


In [6]:
print("QUBO 矩阵前 5 行 5 列:\n", Q[:17, :17])

QUBO 矩阵前 5 行 5 列:
 [[     0.  10000.  10000.  30000.      0.      0.  30000.      0.      0.
  -10000. -20000.  10000.  20000.      0.      0.      0.      0.]
 [ 10000.  80000.  10000.      0.  30000.      0.      0.  90000.      0.
  -30000. -60000.      0.      0.  30000.  60000.      0.      0.]
 [ 10000.  10000.  80000.      0.      0.  90000.      0.      0.  30000.
  -30000. -60000.      0.      0.      0.      0.  30000.  60000.]
 [ 30000.      0.      0.  80000.  10000.  10000.  90000.      0.      0.
  -30000. -60000.  30000.  60000.      0.      0.      0.      0.]
 [     0.  30000.      0.  10000.      0.  10000.      0.  30000.      0.
  -10000. -20000.      0.      0.  10000.  20000.      0.      0.]
 [     0.      0.  90000.  10000.  10000.  80000.      0.      0.  30000.
  -30000. -60000.      0.      0.      0.      0.  30000.  60000.]
 [ 30000.      0.      0.  90000.      0.      0.  80000.  10000.  10000.
  -30000. -60000.  30000.  60000.      0.      0.      0.    

In [8]:
from itertools import product


# 自定义暴力求解函数，考虑 J 和 h
def brute_force_ising(J, h):
    N = len(h)
    min_energy = float('inf')
    ground_spins = None
    for spins in product([-1, 1], repeat=N):
        spins = np.array(spins)
        energy = -0.5 * np.sum(J * np.outer(spins, spins)) - np.dot(h, spins)
        if energy < min_energy:
            min_energy = energy
            ground_spins = spins
    return ground_spins, min_energy

In [25]:
from cim_optimizer.solve_Ising import Ising

test = Ising(J, h).solve(
    use_GPU=True,
    num_runs=50,
    num_timesteps_per_run=5000,
    cac_time_step=0.02,
    cac_r=-0.5,
    cac_alpha=0.8,
    cac_beta=0.3,
    cac_gamma=0.005,
    cac_delta=15,
    cac_mu=0.9,
    cac_rho=1.5,
    cac_tau=200,
    return_spin_trajectories_all_runs=True,
    hyperparameters_randomtune=False
)

External Field Detected
Target Ising Energy: -inf.
Best Ising Energy Found: -612498.5.
Corresponding Spin Configuration: [-1. -1.  1. -1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1.  1.  1.].
Time Elapsed: 522.4227931499481.
Number of Runs Completed: 50.


In [26]:
# 提取 CIM 结果
print("test.result 属性:", [attr for attr in dir(test.result) if not attr.startswith('_')])
best_energy = test.result.lowest_energy
print(f"CIM 最佳能量: {best_energy}")

test.result 属性: ['energies', 'energy_evolution', 'lowest_energy', 'lowest_energy_spin_config', 'result_data', 'spin_config_all_runs', 'spin_trajectories', 'time']
CIM 最佳能量: -612498.5


In [27]:
# 提取自旋配置
best_spins = test.result.lowest_energy_spin_config
print("The spin configuration in the ground state is {}".format(best_spins))

The spin configuration in the ground state is [-1. -1.  1. -1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1.  1.  1.]


In [28]:
# 计算 CIM 解的能量
energy_cim = -0.5 * np.sum(J * np.outer(best_spins, best_spins)) - np.dot(h, best_spins)
print(f"验证 CIM 能量: {energy_cim}")

验证 CIM 能量: -612498.5


In [29]:
# 暴力求解基态
spins_ground, E_ground = brute_force_ising(J, h)
print(f"基态自旋配置: {spins_ground}")
print(f"基态能量: {E_ground}")

基态自旋配置: [-1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1]
基态能量: -862498.5


In [30]:
# 比较 CIM 和基态能量
if abs(energy_cim - E_ground) < 1e-5:
    print("CIM 解达到基态能量")
else:
    print(f"CIM 解与基态能量差: {energy_cim - E_ground}")

CIM 解与基态能量差: 250000.0


In [31]:
# 解码 FOSSP 变量
n, m, l = 3, 3, 2
P = np.array([[1, 3, 3], [3, 1, 3], [3, 3, 1]])
x_sol = [(best_spins[i] + 1) / 2 for i in range(n * m)]
z_sol = [(best_spins[n * m + h] + 1) / 2 for h in range(l)]
tau_sol = [(best_spins[n * m + l + j * l + h] + 1) / 2 for j in range(m) for h in range(l)]

In [32]:
# 计算 C_max
C_max = sum(2 ** h * z_sol[h] for h in range(l))
print(f"最大完工时间 (C_max): {C_max}")

最大完工时间 (C_max): 3.0


In [33]:
# 验证分配约束
x_matrix = np.array(x_sol).reshape(n, m)
print("分配矩阵:\n", x_matrix)
for i in range(n):
    if abs(sum(x_matrix[i]) - 1) > 1e-5:
        print(f"警告：作业 {i + 1} 分配不满足约束，sum(x_{i}j) = {sum(x_matrix[i])}")
    else:
        print(f"作业 {i + 1} 分配满足约束")

分配矩阵:
 [[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 0.]]
作业 1 分配满足约束
作业 2 分配满足约束
警告：作业 3 分配不满足约束，sum(x_2j) = 0.0


In [34]:
# 验证机器完工时间
for j in range(m):
    T_j = sum(x_matrix[i, j] * P[i, j] for i in range(n))
    print(f"机器 {j + 1} 完工时间: {T_j}")
    if T_j > C_max + 1e-5:
        print(f"警告：机器 {j + 1} 完工时间 {T_j} 超过 C_max {C_max}")

机器 1 完工时间: 0.0
机器 2 完工时间: 1.0
机器 3 完工时间: 3.0


In [35]:
# from cim_optimizer.CIM_helper import brute_force
#
# spins_ground, E_ground = brute_force(J)
# print("The spin configuration in the ground state is {}".format(spins_ground))
# print("The ground energy is {}".format(E_ground))

In [36]:
# 调试：检查结果属性
print("test.result 属性:", [attr for attr in dir(test.result) if not attr.startswith('_')])

# 提取最低能量
best_energy = test.result.lowest_energy
print(f"最佳能量: {best_energy}")
# best_spins = spins_ground
# best_spins = test.result.lowest_energy_spin_config
# best_spins = [1,  -1,  -1,  -1,  1,  -1,  -1,  -1, 1,  -1. -1,  1,  1,  1,  1,  1, -1,1]
print(f"最佳自旋配置: {best_spins}")

# 验证自旋长度
if len(best_spins) != N:
    print(f"错误：自旋长度 {len(best_spins)} 不等于预期 {N}")
    raise ValueError("自旋配置无效")

test.result 属性: ['energies', 'energy_evolution', 'lowest_energy', 'lowest_energy_spin_config', 'result_data', 'spin_config_all_runs', 'spin_trajectories', 'time']
最佳能量: -612498.5
最佳自旋配置: [-1. -1.  1. -1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1.  1.  1.]


In [37]:
# 解码解
x_sol = [(best_spins[i] + 1) / 2 for i in range(n * m)]
z_sol = [(best_spins[n * m + h] + 1) / 2 for h in range(l)]
tau_sol = [(best_spins[n * m + l + j * l + h] + 1) / 2 for j in range(m) for h in range(l)]
C_max = sum(2 ** h * z_sol[h] for h in range(l))
print(f"最大完工时间 (C_max): {C_max}")
print(f"分配方案 (x_ij): {x_sol}")

# 验证分配约束
x_matrix = np.array(x_sol).reshape(n, m)
print("分配矩阵:\n", x_matrix)
for i in range(n):
    if abs(sum(x_matrix[i]) - 1) > 1e-5:
        print(f"警告：作业 {i + 1} 分配不满足约束，sum(x_{i}j) = {sum(x_matrix[i])}")
    else:
        print(f"作业 {i + 1} 分配满足约束")

# 验证机器完工时间
for j in range(m):
    T_j = sum(x_matrix[i, j] * P[i, j] for i in range(n))
    print(f"机器 {j + 1} 完工时间: {T_j}")
    if T_j > C_max + 1e-5:
        print(f"警告：机器 {j + 1} 完工时间 {T_j} 超过 C_max {C_max}")

最大完工时间 (C_max): 3.0
分配方案 (x_ij): [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
分配矩阵:
 [[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 0.]]
作业 1 分配满足约束
作业 2 分配满足约束
警告：作业 3 分配不满足约束，sum(x_2j) = 0.0
机器 1 完工时间: 0.0
机器 2 完工时间: 1.0
机器 3 完工时间: 3.0


In [41]:
import kaiwu as kw

splitted_ret, last_idx = kw.preprocess.perform_precision_adaption_split(Q, 4)
print(splitted_ret)

[[0. 7. 7. ... 0. 0. 0.]
 [7. 0. 7. ... 0. 0. 0.]
 [7. 7. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 7. 7.]
 [0. 0. 0. ... 7. 0. 7.]
 [0. 0. 0. ... 7. 7. 0.]]
